In [25]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder


In [16]:

CSV_PATH     = 'C:/Users/sheet/Desktop/KneeRevive/windowed_kneerevive.csv'
SEQ_LEN      = 100
FEATURE_COLS = ['AccelX','AccelY','AccelZ','GyroX','GyroY','GyroZ']
LABEL_COL    = 'Label'


In [17]:
df = pd.read_csv(CSV_PATH)

In [18]:
df.head

<bound method NDFrame.head of           AccelX    AccelY    AccelZ     GyroX     GyroY     GyroZ  WindowID  \
0       0.099343 -0.027653  0.129538  0.076151 -0.011708 -0.011707         0   
1       0.315843  0.153487 -0.093895  0.027128 -0.023171 -0.023286         0   
2       0.048392 -0.382656 -0.344984 -0.028114 -0.050642  0.015712         0   
3      -0.181605 -0.282461  0.293130 -0.011289  0.003376 -0.071237         0   
4      -0.108877  0.022185 -0.230199  0.018785 -0.030032 -0.014585         0   
...          ...       ...       ...       ...       ...       ...       ...   
599995 -0.039331  9.793256 -0.006271 -0.007733  0.021165  0.003815      5999   
599996  0.028379  9.792432  0.021559  0.004022 -0.014029  0.007042      5999   
599997  0.001787  9.853975 -0.017008 -0.027766 -0.007519  0.002651      5999   
599998 -0.018875  9.814159  0.009360 -0.008983 -0.007533 -0.011495      5999   
599999  0.025614  9.816862  0.045889 -0.006905  0.007759  0.003495      5999   

         

In [19]:
#validation
n_rows = len(df)
assert 'WindowID' in df.columns, "Need a WindowID column to group timesteps"
n_windows = df['WindowID'].nunique()
assert n_windows * SEQ_LEN == n_rows, "Rows must equal windows×SEQ_LEN"

In [20]:
# ─── 3. BUILD X ARRAY ─────────────────────────────────────────────────
# Sort by WindowID so timesteps stay in order
df = df.sort_values(['WindowID']).reset_index(drop=True)

X = df[FEATURE_COLS].values \
       .reshape(n_windows, SEQ_LEN, len(FEATURE_COLS)) \
       .astype(np.float32)

In [21]:
# ─── 4. BUILD y VECTOR ─────────────────────────────────────────────────
# If all rows in a window share the same label, .first() works; else use mode()
y = df.groupby('WindowID')[LABEL_COL].first().values

In [22]:
# Encode text labels → integers
le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [27]:
from tsai.all import *


ImportError: Could not import '__path__' from fastcore.dispatch - this module has been moved to the fasttransform package.
To migrate your code, please see the migration guide at: https://answerdotai.github.io/fasttransform/fastcore_migration_guide.html

In [26]:
#─── 5. TRAIN/VALID SPLIT ──────────────────────────────────────────────
splits = get_splits(y_encoded, valid_size=0.2, stratify=True)

NameError: name 'get_splits' is not defined

In [ ]:
# ─── 6. DATALOADERS ───────────────────────────────────────────────────
dls = get_ts_dls(X, y_encoded, splits=splits)


In [ ]:
# ─── 7. LEARNER & FINE-TUNE ───────────────────────────────────────────
learn = ts_learner(dls, InceptionTime, pretrained=True)
learn.fine_tune(5)

In [ ]:
# ─── 8. RESULTS ────────────────────────────────────────────────────────
learn.show_results(max_n=6)


In [ ]:
# ─── 9. SAVE MODEL ────────────────────────────────────────────────────
learn.export('kneerevive_inceptiontime.pkl')


In [ ]:
# ───  🔍 PRINT METRICS ─────────────────────────────────────────────────
metrics = learn.validate()
print(f"\nValidation loss: {metrics[0]:.4f}, accuracy: {metrics[1]:.4f}")
